In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
import json 
import numpy as np

# The Data Sets - Abalone 

In [ ]:
abalones_df = pd.read_csv('COMP472-A1-datasets/abalone.csv')
# print(penguins_df.head())

In [ ]:
# print csv file info
print(abalones_df.info()) 

In [ ]:
# # label encoding string values to int
le = preprocessing.LabelEncoder()
abalones_df['Type'] = le.fit_transform(abalones_df['Type'])

print(abalones_df.info()) 
print(abalones_df.head())

In [ ]:
# converting the features into 1-hot vectors
# abalones_df = pd.get_dummies(abalones_df, columns=['Type'], prefix='Type', dtype='int64')
# print(abalones_df.info()) 
# print(abalones_df.head())

# Note we've left this commented out to not affect columns of the table being used for the rest of the results

In [ ]:
# plot pie chart based on output class species
total = abalones_df['Type'].count()

num_females = abalones_df[abalones_df['Type'] == 0]['Type'].count()
percent_females= num_females / total * 100

num_infants = abalones_df[abalones_df['Type'] == 1]['Type'].count()
percent_infants = num_infants / total * 100

num_males = abalones_df[abalones_df['Type'] == 2]['Type'].count()
percent_males = num_males / total * 100

species_percentages = [percent_females, percent_males, percent_infants]

plt.figure(figsize=(7, 7))
plt.pie(species_percentages, labels=['Females', 'Males', 'Infants'], autopct='%1.1f%%')
plt.title("Percentange of Each Abalone Type")
plt.savefig('ablone_types_pie_chart.png')

plt.show()

# Note: dataset fairly balanced in terms of abalone types

# balanced dataset : accuracy 

In [ ]:
# split data into training and testing sets
# default split is 25% testing, 75% training
# data is shuffled by default, but no seeding applied 

X, y = [abalones_df.drop('Type', axis=1), abalones_df['Type']]

X_train_set, X_test_set, y_train_set, y_test_set = train_test_split(X, y)

### Generate Performance Measures File

In [ ]:
with open('abalone-performance.txt', 'w') as f:
    f.write('Performance Measures \n\n\n')

In [ ]:
with open('abalone-performance_5_runs.txt', 'w') as f:
    f.write('Performance Measures \n\n\n')

# Base-DT
=> a decision tree with the default parameters 

In [ ]:
# default parameter for criterion = Gini impurity 
dtc = tree.DecisionTreeClassifier()

dtc.fit(X_train_set, y_train_set)
tree.plot_tree(dtc, max_depth=6)

dot_data = tree.export_graphviz(dtc, out_file=None,
    feature_names= ['LongestShell', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings'],
    class_names=['F','M','I'],
    filled=True, 
    rounded=True,
    max_depth=6,)
graph = graphviz.Source(dot_data) 

graph.render("abalone_types_base_dt")   # save to pdf

In [ ]:
y_predict = dtc.predict(X_test_set)

print(X_test_set)
print("Predicted output: ", le.inverse_transform(y_predict))

# Base-DT Performance Measures 

In [ ]:
with open('abalone-performance.txt', 'a') as f:
    f.write('**********************************************************\n\n(A) Base-DT Performance Measures \n\n')
    f.write('Hyperparameters: \n \t- criterion: Gini impurity \n\t- max_depth: 6 \n\t- min_samples_split: 2 \n\n')
    f.write('\n(B) Confusion Matrix: \n')
    f.write(str(confusion_matrix(y_test_set, y_predict)))
    f.write('\n\n(C - D) Classification Report: \n')
    f.write(str(classification_report(y_test_set, y_predict)))
    f.write('\n(C - D) Accuracy Score: ')
    f.write(str(accuracy_score(y_test_set, y_predict)))

Re-run the Base-DT code above 5 times. 

In [ ]:
base_dt_accuracy_scores = []
base_dt_f1_scores_macro_avgs = []
base_dt_f1_scores_weighted_avgs = []

for i in range(5):
    # default parameter for criterion = Gini impurity 
    dtc = tree.DecisionTreeClassifier()

    dtc.fit(X_train_set, y_train_set)
    tree.plot_tree(dtc, max_depth=6)

    dot_data = tree.export_graphviz(dtc, out_file=None,
        feature_names= ['LongestShell', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings'],
        class_names=['F','M','I'],
        filled=True, 
        rounded=True,
        max_depth=6,)
    graph = graphviz.Source(dot_data) 

    # graph.render("abalone_types_base_dt")   # save to pdf

    y_predict = dtc.predict(X_test_set)

    accuracy = accuracy_score(y_test_set, y_predict)
    base_dt_accuracy_scores.append(accuracy)
    
    f1_score_macro_avg = f1_score(y_test_set, y_predict, average='macro')
    base_dt_f1_scores_macro_avgs.append(f1_score_macro_avg)
    
    f1_score_weighted_avg = f1_score(y_test_set, y_predict, average='weighted')
    base_dt_f1_scores_weighted_avgs.append(f1_score_weighted_avg)
    
    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))
    with open('abalone-performance_5_runs.txt', 'a') as f:
        f.write(f'\n\n**********************************************************\n\nRUN {i + 1} \n\n(A) Base-DT Performance Measures \n\n')
        f.write('Hyperparameters: \n \t- criterion: Gini impurity \n\t- max_depth: None \n\t- min_samples_split: 2 \n\n')
        f.write('\n(B) Confusion Matrix: \n')
        f.write(str(confusion_matrix(y_test_set, y_predict)))
        f.write('\n\n(C - D) Classification Report: \n')
        f.write(str(classification_report(y_test_set, y_predict)))
        f.write('\n(C - D) Accuracy Score: ')
        f.write(str(accuracy))

# Top-DT
=> a Decision Tree found using a gridsearch 

In [ ]:
# create decision tree model
dtc_top_dt = tree.DecisionTreeClassifier()

# define hyperparameters to test for best model
hyperparameters = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 6, 2],
    'min_samples_split': [35, 150, 300],
}

# use grid search to find best hyperparameters
gs = GridSearchCV(dtc_top_dt, hyperparameters, verbose=1)
gs.fit(X_train_set, y_train_set)
best_hyperparameters = gs.best_params_

print('Best hyperparameters:\n', best_hyperparameters)

# create decision tree model with best hyperparameters
dtc_top_dt_best_params = tree.DecisionTreeClassifier(**best_hyperparameters)
# train model with best hyperparameters
dtc_top_dt_best_params.fit(X_train_set, y_train_set)

# plot decision tree
tree.plot_tree(dtc_top_dt_best_params)

dot_data = tree.export_graphviz(dtc_top_dt_best_params, out_file=None,
    feature_names= ['LongestShell', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings'],
    class_names=['F','M','I'],
    filled=True, 
    rounded=True,
)
graph = graphviz.Source(dot_data) 

graph.render("abalone_types_top_dt")   # save to pdf

In [ ]:
y_predict = dtc_top_dt_best_params.predict(X_test_set)

print(X_test_set)
print("Predicted output: ", le.inverse_transform(y_predict))

# Top-DT Performance Measures 

In [ ]:
with open('abalone-performance.txt', 'a') as f:
    f.write('\n\n\n**********************************************************\n\n(A) Top-DT Performance Measures \n\n')
    f.write("Hyperparameters: " + str(json.dump(best_hyperparameters, f)) + "\n\n")
    f.write('\n(B) Confusion Matrix: \n')
    f.write(str(confusion_matrix(y_test_set, y_predict)))
    f.write('\n\n(C - D) Classification Report: \n')
    f.write(str(classification_report(y_test_set, y_predict)))
    f.write('\n(C - D) Accuracy Score: ')
    f.write(str(accuracy_score(y_test_set, y_predict)))

Re-run the Top-DT code above 5 times. 

In [ ]:
top_dt_accuracy_scores = []
top_dt_f1_scores_macro_avgs = []
top_dt_f1_scores_weighted_avgs = []

for i in range(5):
    # create decision tree model
    dtc_top_dt = tree.DecisionTreeClassifier()

    # define hyperparameters to test for best model
    hyperparameters = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 6, 2],
        'min_samples_split': [35, 150, 300],
    }

    # use grid search to find best hyperparameters
    gs = GridSearchCV(dtc_top_dt, hyperparameters, verbose=1)
    gs.fit(X_train_set, y_train_set)
    best_hyperparameters = gs.best_params_

    print('Best hyperparameters:\n', best_hyperparameters)

    # create decision tree model with best hyperparameters
    dtc_top_dt_best_params = tree.DecisionTreeClassifier(**best_hyperparameters)
    # train model with best hyperparameters
    dtc_top_dt_best_params.fit(X_train_set, y_train_set)

    # plot decision tree
    tree.plot_tree(dtc_top_dt_best_params)

    dot_data = tree.export_graphviz(dtc_top_dt_best_params, out_file=None,
        feature_names= ['LongestShell', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings'],
        class_names=['F','M','I'],
        filled=True, 
        rounded=True,
    )
    graph = graphviz.Source(dot_data) 

    # graph.render("abalone_types_top_dt")   # save to pdf
    
    y_predict = dtc_top_dt_best_params.predict(X_test_set)

    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))

    accuracy = accuracy_score(y_test_set, y_predict)
    top_dt_accuracy_scores.append(accuracy)
    
    f1_score_macro_avg = f1_score(y_test_set, y_predict, average='macro')
    top_dt_f1_scores_macro_avgs.append(f1_score_macro_avg)
    
    f1_score_weighted_avg = f1_score(y_test_set, y_predict, average='weighted')
    top_dt_f1_scores_weighted_avgs.append(f1_score_weighted_avg)
    
    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))
    with open('abalone-performance_5_runs.txt', 'a') as f:
        f.write(f'\n\n\n**********************************************************\n\nRUN {i + 1} \n\n(A) Top-DT Performance Measures \n\n')
        f.write("Hyperparameters: " + str(json.dump(best_hyperparameters, f)) + "\n\n")
        f.write('\n(B) Confusion Matrix: \n')
        f.write(str(confusion_matrix(y_test_set, y_predict)))
        f.write('\n\n(C - D) Classification Report: \n')
        f.write(str(classification_report(y_test_set, y_predict)))
        f.write('\n(C - D) Accuracy Score: ')
        f.write(str(accuracy))

# Base-MLP
=> a Multi-Layered Perceptron with 2 hidden layers of 100+100 neurons, sigmoid/logistic as activation function, stochastic gradient descent, and default values for the rest of the parameters

In [ ]:
# activation = 'logistic' => sigmoid function
# solver = 'sgd' => stochastic gradient descent
# default max_iter = 200 (number of epochs)
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), activation='logistic', solver='sgd')
mlp.fit(X_train_set, y_train_set)

In [ ]:
y_predict = mlp.predict(X_test_set)
print(X_test_set)
print("Predicted output: ", le.inverse_transform(y_predict))

# Base-MLP Performance Measures 

In [ ]:
with open('abalone-performance.txt', 'a') as f:
    f.write('\n\n\n**********************************************************\n\n(A) Base-MLP Performance Measures \n\n')
    f.write('Hyperparameters: \n \t- hidden_layer_sizes: (100, 100) \n\t- activation: logistic \n\t- solver: sgd \n\t- max_iter = 200 \n\t- shuffle = true \n\n')
    f.write('\n(B) Confusion Matrix: \n')
    f.write(str(confusion_matrix(y_test_set, y_predict)))
    f.write('\n\n(C - D) Classification Report: \n')
    f.write(str(classification_report(y_test_set, y_predict)))
    f.write('\n(C - D) Accuracy Score: ')
    f.write(str(accuracy_score(y_test_set, y_predict)))

Re-run the Base-MLP code above 5 times. 

In [ ]:
base_mlp_accuracy_scores = []
base_mlp_f1_scores_macro_avgs = []
base_mlp_f1_scores_weighted_avgs = []


for i in range(5):
    # activation = 'logistic' => sigmoid function
    # solver = 'sgd' => stochastic gradient descent
    # default max_iter = 200 (number of epochs)
    mlp = MLPClassifier(hidden_layer_sizes=(100, 100), activation='logistic', solver='sgd')
    mlp.fit(X_train_set, y_train_set)
    
    y_predict = dtc_top_dt_best_params.predict(X_test_set)

    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))

    accuracy = accuracy_score(y_test_set, y_predict)
    base_mlp_accuracy_scores.append(accuracy)
    
    f1_score_macro_avg = f1_score(y_test_set, y_predict, average='macro')
    base_mlp_f1_scores_macro_avgs.append(f1_score_macro_avg)
    
    f1_score_weighted_avg = f1_score(y_test_set, y_predict, average='weighted')
    base_mlp_f1_scores_weighted_avgs.append(f1_score_weighted_avg)
    
    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))
    
    with open('abalone-performance_5_runs.txt', 'a') as f:
        f.write(f'\n\n\n**********************************************************\n\nRUN {i + 1} \n\n(A) Base-MLP Performance Measures \n\n')
        f.write('Hyperparameters: \n \t- hidden_layer_sizes: (100, 100) \n\t- activation: logistic \n\t- solver: sgd \n\t- max_iter = 200 \n\t- shuffle = true \n\n')
        f.write('\n(B) Confusion Matrix: \n')
        f.write(str(confusion_matrix(y_test_set, y_predict)))
        f.write('\n\n(C - D) Classification Report: \n')
        f.write(str(classification_report(y_test_set, y_predict)))
        f.write('\n(C - D) Accuracy Score: ')
        f.write(str(accuracy))

# Top-MLP
=> a Multi-Layered Perceptron found using grid search

In [ ]:
mlp_top = MLPClassifier()


# define hyperparameters to test for best model
hyperparameters = {
    'activation': ['logistic', 'tanh', 'relu'],
    'hidden_layer_sizes': [(100, 150), (200, 200, 200)],
    'solver': ['adam', 'sgd'],
    'max_iter': [1500],
}

# use grid search to find best hyperparameters
gs = GridSearchCV(mlp_top, hyperparameters, verbose=1)
gs.fit(X_train_set, y_train_set)
best_hyperparameters = gs.best_params_

print('Best hyperparameters:\n', best_hyperparameters)

# create decision tree model with best hyperparameters
mlp_top_best_params = MLPClassifier(**best_hyperparameters)

# train model with best hyperparameters
mlp_top_best_params.fit(X_train_set, y_train_set)

In [ ]:
y_predict = mlp_top_best_params.predict(X_test_set)

print(X_test_set)
print("Predicted output: ", le.inverse_transform(y_predict))

## Top-MLP Performance Measures

In [ ]:
with open('abalone-performance.txt', 'a') as f:
    f.write('\n\n\n**********************************************************\n\n(A) Top-MLP Performance Measures \n\n')
    f.write("Hyperparameters: " + str(json.dump(best_hyperparameters, f)) + "\n\n")
    f.write('\n(B) Confusion Matrix: \n')
    f.write(str(confusion_matrix(y_test_set, y_predict)))
    f.write('\n\n(C - D) Classification Report: \n')
    f.write(str(classification_report(y_test_set, y_predict)))
    f.write('\n(C - D) Accuracy Score: ')
    f.write(str(accuracy_score(y_test_set, y_predict)))

Re-run the Top-MLP code above 5 times. 

In [ ]:
top_mlp_accuracy_scores = []
top_mlp_f1_scores_macro_avgs = []
top_mlp_f1_scores_weighted_avgs = []


for i in range(5):
    mlp_top = MLPClassifier()


    # define hyperparameters to test for best model
    hyperparameters = {
        'activation': ['logistic', 'tanh', 'relu'],
        'hidden_layer_sizes': [(100, 150), (200, 200, 200)],
        'solver': ['adam', 'sgd'],
        'max_iter': [1500],
    }

    # use grid search to find best hyperparameters
    gs = GridSearchCV(mlp_top, hyperparameters, verbose=1)
    gs.fit(X_train_set, y_train_set)
    best_hyperparameters = gs.best_params_

    print('Best hyperparameters:\n', best_hyperparameters)

    # create decision tree model with best hyperparameters
    mlp_top_best_params = MLPClassifier(**best_hyperparameters)

    # train model with best hyperparameters
    mlp_top_best_params.fit(X_train_set, y_train_set)
        
    y_predict = dtc_top_dt_best_params.predict(X_test_set)

    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))

    accuracy = accuracy_score(y_test_set, y_predict)
    top_mlp_accuracy_scores.append(accuracy)
    
    f1_score_macro_avg = f1_score(y_test_set, y_predict, average='macro')
    top_mlp_f1_scores_macro_avgs.append(f1_score_macro_avg)
    
    f1_score_weighted_avg = f1_score(y_test_set, y_predict, average='weighted')
    top_mlp_f1_scores_weighted_avgs.append(f1_score_weighted_avg)
    
    print(X_test_set)
    print("Predicted output: ", le.inverse_transform(y_predict))
    
    with open('penguin-performance_5_runs.txt', 'a') as f:
        f.write(f'\n\n\n**********************************************************\n\nRUN {i + 1} \n\n(A) Top-MLP Performance Measures \n\n')
        f.write("Hyperparameters: " + str(json.dump(best_hyperparameters, f)) + "\n\n")
        f.write('\n(B) Confusion Matrix: \n')
        f.write(str(confusion_matrix(y_test_set, y_predict)))
        f.write('\n\n(C - D) Classification Report: \n')
        f.write(str(classification_report(y_test_set, y_predict)))
        f.write('\n(C - D) Accuracy Score: ')
        f.write(str(accuracy))

# Summarize Overall Averages of the 5 Runs 

In [ ]:
with open('abalone-performance_5_runs.txt', 'a') as f:
        f.write(f'\n\n\n**********************************************************\n\nOverall Performance Measure Averages of the 5 Runs \n\n')
       
        f.write('\n\n\n*** Base-DT ***')
        
        f.write('\nBase-DT Average Accuracy Score: ' + str(np.mean(base_dt_accuracy_scores)))
        f.write('\nBase-DT Average Accuracy Score Standard Deviation: ' + str(np.std(base_dt_accuracy_scores)))
        f.write('\nBase-DT Average Accuracy Score Variance: ' + str(np.var(base_dt_accuracy_scores)))
        
        f.write('\n\nBase-DT Average F1 Score (Macro): ' + str(np.mean(base_dt_f1_scores_macro_avgs)))
        f.write('\nBase-DT Average F1 Score (Macro) Standard Deviation: ' + str(np.std(base_dt_f1_scores_macro_avgs)))
        f.write('\nBase-DT Average F1 Score (Macro) Variance: ' + str(np.var(base_dt_f1_scores_macro_avgs)))
        
        
        f.write('\n\nBase-DT Average F1 Score (Weighted): ' + str(np.mean(base_dt_f1_scores_weighted_avgs)))
        f.write('\nBase-DT Average F1 Score (Weighted) Standard Deviation: ' + str(np.std(base_dt_f1_scores_weighted_avgs)))
        f.write('\nBase-DT Average F1 Score (Weighted) Variance: ' + str(np.var(base_dt_f1_scores_weighted_avgs)))
        
        
        
        f.write('\n\n\n*** Top-DT ***')
        
        f.write('\n\n\nTop-DT Average Accuracy Score: ' + str(np.mean(top_dt_accuracy_scores)))
        f.write('\nTop-DT Average Accuracy Score Standard Deviation: ' + str(np.std(top_dt_accuracy_scores)))
        f.write('\nTop-DT Average Accuracy Score Variance: ' + str(np.var(top_dt_accuracy_scores)))
        
        f.write('\n\nTop-DT Average F1 Score (Macro): ' + str(np.mean(top_dt_f1_scores_macro_avgs)))
        f.write('\nTop-DT Average F1 Score (Macro) Standard Deviation: ' + str(np.std(top_dt_f1_scores_macro_avgs)))
        f.write('\nTop-DT Average F1 Score (Macro) Variance: ' + str(np.var(top_dt_f1_scores_macro_avgs)))
        
        f.write('\n\nTop-DT Average F1 Score (Weighted): ' + str(np.mean(top_dt_f1_scores_weighted_avgs)))
        f.write('\nTop-DT Average F1 Score (Weighted) Standard Deviation: ' + str(np.std(top_dt_f1_scores_weighted_avgs)))
        f.write('\nTop-DT Average F1 Score (Weighted) Variance: ' + str(np.var(top_dt_f1_scores_weighted_avgs)))
        
        
        
        f.write('\n\n\n*** Base-MLP ***')
        
        f.write('\n\n\nBase-MLP Average Accuracy Score: ' + str(np.mean(base_mlp_accuracy_scores)))
        f.write('\nBase-MLP Average Accuracy Score Standard Deviation: ' + str(np.std(base_mlp_accuracy_scores)))
        f.write('\nBase-MLP Average Accuracy Score Variance: ' + str(np.var(base_mlp_accuracy_scores)))
        
        f.write('\n\nBase-MLP Average F1 Score (Macro): ' + str(np.mean(base_mlp_f1_scores_macro_avgs)))
        f.write('\nBase-MLP Average F1 Score (Macro) Standard Deviation: ' + str(np.std(base_mlp_f1_scores_macro_avgs)))
        f.write('\nBase-MLP Average F1 Score (Macro) Variance: ' + str(np.var(base_mlp_f1_scores_macro_avgs)))
        
        f.write('\n\nBase-MLP Average F1 Score (Weighted): ' + str(np.mean(base_mlp_f1_scores_weighted_avgs)))
        f.write('\nBase-MLP Average F1 Score (Weighted) Standard Deviation: ' + str(np.std(base_mlp_f1_scores_weighted_avgs)))
        f.write('\nBase-MLP Average F1 Score (Weighted) Variance: ' + str(np.var(base_mlp_f1_scores_weighted_avgs)))
        
        
        
        f.write('\n\n\n*** Top-MLP ***')
        
        f.write('\n\n\nTop-MLP Average Accuracy Score: ' + str(np.mean(top_mlp_accuracy_scores)))
        f.write('\nTop-MLP Average Accuracy Score Standard Deviation: ' + str(np.std(top_mlp_accuracy_scores)))
        f.write('\nTop-MLP Average Accuracy Score Variance: ' + str(np.var(top_mlp_accuracy_scores)))
        
        f.write('\n\nTop-MLP Average F1 Score (Macro): ' + str(np.mean(top_mlp_f1_scores_macro_avgs)))
        f.write('\nTop-MLP Average F1 Score (Macro) Standard Deviation: ' + str(np.std(top_mlp_f1_scores_macro_avgs)))
        f.write('\nTop-MLP Average F1 Score (Macro) Variance: ' + str(np.var(top_mlp_f1_scores_macro_avgs)))
        
        f.write('\n\nTop-MLP Average F1 Score (Weighted): ' + str(np.mean(top_mlp_f1_scores_weighted_avgs)))
        f.write('\nTop-MLP Average F1 Score (Weighted) Standard Deviation: ' + str(np.std(top_mlp_f1_scores_weighted_avgs)))
        f.write('\nTop-MLP Average F1 Score (Weighted) Variance: ' + str(np.var(top_mlp_f1_scores_weighted_avgs)))